In [5]:
#!pip install langchain-ollama langchain-community==0.2.4 langchain==0.2.3 faiss-cpu==1.8.0 unstructured==0.14.5 unstructured[pdf]==0.14.5 transformers==4.41.2 sentence-transformers==3.0.1

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3:latest")

chain = prompt | model

chain.invoke({"question": "Write System Verilog code for 8-bit Full Adder."})

"A great problem!\n\nTo write the SystemVerilog code for an 8-bit Full Adder, we'll follow these steps:\n\n1. Define the inputs and outputs of the full adder:\n\t* `a`: 8-bit input (A)\n\t* `b`: 8-bit input (B)\n\t* `cin`: 1-bit input (carry-in)\n\t* `s`: 8-bit output (sum)\n\t* `cout`: 1-bit output (carry-out)\n2. Declare the variables needed for the full adder:\n\t* `temp_sum`: temporary sum variable\n3. Implement the logic for each bit position of the full adder:\n\n```\nmodule full_adder #(\n    parameter WIDTH = 8 // width of the full adder\n) (\n    input [WIDTH-1:0] a, b, // inputs A and B\n    input cin, // carry-in\n    output reg [WIDTH-1:0] s, // sum\n    output reg cout // carry-out\n);\n\nalways @(*) begin\n    s = a + b + cin; // calculate the sum\n\n    // generate the carry-out (cout)\n    cout = (a[WIDTH-1] & b[WIDTH-1]) | (a[WIDTH-1] & cin) | (b[WIDTH-1] & cin);\nend\n```\n\nIn this code:\n\n* We define a parameter `WIDTH` to specify the width of the full adder.\n* Th

In [10]:
import ollama
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': 'Write System Verilog code for 8-bit Full Adder.',
  },
])
print(response['message']['content'])

Here is the SystemVerilog code for an 8-bit full adder:
```
module full_adder_8bit(input logic [7:0] a, input logic [7:0] b, input logic carry_in,
                       output logic [7:0] sum, output logic carry_out);

logic [7:0] t_sum;
logic t_carry;

// Bit-wise addition
assign {t_carry, t_sum[7:1]} = a + b + carry_in;

// Carry-out calculation
assign carry_out = t_carry;

// Sum calculation
assign sum = t_sum;

endmodule
```
Let me explain what's going on in this code:

* The module is called `full_adder_8bit`, and it takes three inputs:
	+ `a` and `b`: 8-bit vectors representing the two input values to be added.
	+ `carry_in`: a single bit indicating whether there is a carry-in from the previous stage (0 or 1).
* The module produces two outputs:
	+ `sum`: an 8-bit vector representing the sum of the inputs and any carry-in.
	+ `carry_out`: a single bit indicating whether there is a carry-out to the next stage (0 or 1).
* The code uses bit-wise addition to calculate the sum and car

In [ ]:
import os

from langchain_community.llms import Ollama
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

In [ ]:
# loading the LLM
llm = Ollama(
    model="llama3:latest",
    temperature=0
)

In [ ]:
# loading the document
loader = UnstructuredFileLoader("NIPS-2017-attention-is-all-you-need-Paper.pdf")
documents = loader.load()

In [ ]:
# create document chunks
text_splitter = CharacterTextSplitter(separator="/n",
                                      chunk_size=1000,
                                      chunk_overlap=200)

In [ ]:
# loading the vector embedding model
embeddings = HuggingFaceEmbeddings()

In [ ]:
knowledge_base = FAISS.from_documents(text_chunks, embeddings)

In [ ]:
# retrieval QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=knowledge_base.as_retriever())

In [ ]:
question = "What is this document about?"
response = qa_chain.invoke({"query": question})
print(response["result"])

In [ ]:
question = "What is the architecture discussed in the model?"
response = qa_chain.invoke({"query": question})
print(response["result"])